In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [2]:
df = pd.read_excel(r'/Users/mei/Documents/project/athleisure_ethics_dataset.xlsx')

In [3]:
df

,brand_name,parent_ company,notes,message_text,message_page_name,message_link,message_collected_data,good_on_you_date_sourced,pricepoint,overall_ethic_rating,good_on_you _link,Twitter Followers,FB Page Likes (total),IG Followers,pinterest,social_media_numbers date sourced,"# social media accounts on website (youtube, facebook, twitter, instagram, pinterest, tiktok, linkedin, strava, vimeo, tumbler, spotify, snapchat or weibo)",notes (optional),get a number of different accounts posted; might need to hover to get the real number
0,girlfriend collective,NaN,NaN,When we started Girlfriend Collective our firs...,about,https://www.girlfriend.com/pages/about,2021-05-22,2021-05-12,$$,great,https://directory.goodonyou.eco/brand/girlfrie...,364,75768.0,488424.0,826.0,2021-06-05,2,NaN,NaN
1,organic basics,NaN,NaN,"To make any meaningful change in our industry,...",activism,https://impact-report.organicbasics.com/activism,2021-05-22,2021-05-12,$$,great,https://directory.goodonyou.eco/brand/organic-...,1481,36137.0,292896.0,1202.0,2021-06-05,3,NaN,NaN
2,beaumont organic,NaN,NaN,Organic cotton remains at the centre of our co...,our sustainable,https://www.beaumontorganic.com/pages/our-sust...,2021-05-22,2021-05-12,$$,good,https://directory.goodonyou.eco/brand/beaumont...,1678,32038.0,25475.0,543.0,2021-06-05,4,NaN,NaN
3,threads 4 thought,NaN,NaN,A Simple Choice. We founded Threads 4 Thought ...,about,https://www.threads4thought.com/pages/about-th...,2021-05-22,2021-05-12,$$,good,https://directory.goodonyou.eco/brand/threads-...,2929,53915.0,28956.0,5737.0,2021-06-05,3,NaN,NaN
4,adidas by stella mccartney,adidas,NaN,ADIDAS BY STELLA MCCARTNEY: SHAPING A MORE SUS...,blog,https://www.adidas.com/us/blog/392090-adidas-b...,2021-05-22,2021-05-12,$$,good,https://directory.goodonyou.eco/brand/adidas-b...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,altar'd state,NaN,"might remove, nothing about Sustainability",What does Stand Out. For Good mean? Altar'd St...,FAQ,https://www.altardstate.com/help/?page=faqs-page,2021-06-01,2021-05-12,$$,we avoid,https://directory.goodonyou.eco/brand/altard-s...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
246,starter,NaN,"might remove, nothing about Sustainability",AUTHENTIC SINCE 1971\nA premium athletic brand...,About us,https://starter.com/aboutus/,2021-06-01,2021-05-12,$$,we avoid,https://directory.goodonyou.eco/brand/starter,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
247,sharper image,NaN,couldn't find this brand anywhere online,NaN,NaN,NaN,2021-06-01,2021-05-12,$$,we avoid,https://directory.goodonyou.eco/brand/sharper-...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
248,danskin,NaN,"might remove, nothing about Sustainability",We are the dance of life\nWe’ve been the autho...,About us,https://danskin.com/pages/about-us,2021-06-01,2021-05-12,$$,we avoid,https://directory.goodonyou.eco/brand/danskin,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


### Functions

In [4]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [5]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        
        expanded_contraction = first_char + expanded_contraction[1:]
        
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    
    return expanded_text

In [6]:
def remove_stopwords(tokens, is_lower_case = True):
    
    #tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    
    return filtered_text

### Remove nulls & create new table for nulls

In [7]:
df = df[df['message_text'].notnull()]
df_null = df[df['message_text'].isnull()]

### Remove new lines

In [8]:
df = df.replace(r'\n','. ', regex=True) 

In [9]:
df['message_text'][245]

"What does Stand Out. For Good mean? Altar'd State started with a mission to fill a gap in the retail industry and change the world for the better. We want to stand out for good in our local communities and around the world by partnering with non-profit organizations helping those in need. We support these organizations with donations (made possible by your Altar'd State and A'Beautiful Soul purchases) and volunteering. How does Altar'd State choose the non-profit organizations/How does my organization become a recipient? Each season, Altar'd State identifies a cause to support, then each store location chooses a local non-profit organization to fit that cause. 10% of that store's net profits from each Monday (we call it Mission Monday) during the season are donated to the selected organization. The store's associates are also encouraged to volunteer with the non-profit and to welcome the non-profit in their store for meet-and-greet opportunities with our guests. Contact your local sto

###  Sentence count

In [10]:
df['message_text_sentence_count'] = df['message_text'].apply(sent_tokenize).tolist()
df['message_text_sentence_count'] = df['message_text_sentence_count'].apply(len)

## Expand contractions

In [11]:
df['message_text_preprocessed'] = df['message_text'].apply(str)
df['message_text_preprocessed'] = df['message_text_preprocessed'].apply(expand_contractions)

### Lowercasing

In [12]:
df['message_text_preprocessed'] = df['message_text_preprocessed'].str.lower()

### Removing punctuation and noise

In [13]:
# remove unnecessary punctuation
tokenizer = nltk.RegexpTokenizer(r"\w+")
df['message_text_preprocessed'] = df['message_text_preprocessed'].apply(tokenizer.tokenize)

In [14]:
# add them back into a long string
df['message_text_preprocessed'] = df['message_text_preprocessed'].str.join(" ")

### Tokenization

In [15]:
df['tokenized'] = df['message_text_preprocessed'].apply(word_tokenize)

### Remove stop words

In [16]:
nltk.download('stopwords')
stopword_list = stopwords.words('english')
df['tokenized'] = df['tokenized'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /Users/mei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
df['tokenized'][245]

'stand good mean altard state started mission fill gap retail industry change world better want stand good local communities around world partnering non profit organizations helping need support organizations donations made possible altard state abeautiful soul purchases volunteering altard state choose non profit organizations organization become recipient season altard state identifies cause support store location chooses local non profit organization fit cause 10 stores net profits monday call mission monday season donated selected organization stores associates also encouraged volunteer non profit welcome non profit store meet greet opportunities guests contact local store find non profit organization involved find store listing find organizations support stores choose non profit partners associates local altar state beautiful soul best source information home office level also partnership organization called coprodeli usa partnership allows us build schools sponsor students impove

### Word count for each company message text

In [18]:
df['message_text_words_count'] = df['message_text'].str.count(' ') + 1
df['token_words_count'] = df['tokenized'].str.count(' ') + 1

In [19]:
df.sort_values(by = 'token_words_count')

,brand_name,parent_ company,notes,message_text,message_page_name,message_link,message_collected_data,good_on_you_date_sourced,pricepoint,overall_ethic_rating,...,pinterest,social_media_numbers date sourced,"# social media accounts on website (youtube, facebook, twitter, instagram, pinterest, tiktok, linkedin, strava, vimeo, tumbler, spotify, snapchat or weibo)",notes (optional),get a number of different accounts posted; might need to hover to get the real number,message_text_sentence_count,message_text_preprocessed,tokenized,message_text_words_count,token_words_count
215,good american,NaN,"might remove, most words were part of poster/p...","We started with denim, because it’s where we c...",Sustainability,https://www.goodamerican.com/pages/sustainabil...,2021-06-01,2021-05-12,$$,we avoid,...,NaN,NaT,NaN,NaN,NaN,1,we started with denim because it s where we ca...,started denim biggest impact,13,4
162,la senza,NaN,"might remove, nothing about sustainability",WE ARE LA SENZA. LA SENZA IS THE GLOBAL DESTIN...,About us,https://www.lasenza.com/us/about-la-senza.html,2021-06-02,2021-05-12,$$,not good enough,...,NaN,NaT,NaN,NaN,NaN,8,we are la senza la senza is the global destina...,la senza la senza global destination hottest f...,49,28
217,versace,NaN,NaN,"On the 50th anniversary of Earth Day, Versace,...",Sustainability,https://www.versace.com/us/en-us/world-of-vers...,2021-06-01,2021-05-12,$$$,not good enough,...,NaN,NaT,NaN,NaN,NaN,3,on the 50th anniversary of earth day versace a...,50th anniversary earth day versace part capri ...,43,29
188,under armour,NaN,NaN,SUSTAINABILITY AT UNDER ARMOUR. BETTER ATHLETE...,Sustainability,https://about.underarmour.com/community/sustai...,NaT,2021-05-12,$$,not good enough,...,NaN,NaT,NaN,NaN,NaN,6,sustainability at under armour better athletes...,sustainability armour better athletes better w...,68,32
107,gap,NaN,NaN,"Sustainability. . When our founders, Doris and...",Sustainability,https://www.gapinc.com/en-us/values/sustainabi...,2021-05-31,2021-05-12,$$,it's a start,...,359908.0,2021-06-08,0.0,NaN,NaN,4,sustainability when our founders doris and don...,sustainability founders doris fisher started g...,69,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,factorie,cotton on,NaN,Factorie & The Cotton On Group. . Responsibly ...,ethics,https://factorie.com.au/ethics.html,2021-06-01,2021-05-12,$$,not good enough,...,121.0,2021-06-08,3.0,NaN,NaN,88,factorie the cotton on group responsibly made ...,factorie cotton group responsibly made busines...,2148,1206
190,american eagle,NaN,NaN,"Doing the right thing, continually innovating ...",Sustainability,https://www.aeo-inc.com/sustainability/,NaT,2021-05-12,$$,not good enough,...,NaN,NaT,NaN,NaN,NaN,109,doing the right thing continually innovating a...,right thing continually innovating caring glob...,2030,1278
141,rei,NaN,NaN,Product Impact and Circular Commerce. The gear...,sustainable product practices,https://www.rei.com/stewardship/sustainable-pr...,2021-06-01,2021-05-12,$$,not good enough,...,183764.0,2021-06-08,5.0,NaN,NaN,145,product impact and circular commerce the gear ...,product impact circular commerce gear design d...,2541,1538
111,asos,NaN,NaN,THE RESPONSIBILITY. SUSTAINABLE SOURCING PROGR...,Corporate Repsonsibility,https://www.asosplc.com/corporate-responsibili...,2021-05-31,2021-05-12,$$,not good enough,...,1142517.0,2021-06-08,3.0,NaN,NaN,130,the responsibility sustainable sourcing progra...,responsibility sustainable sourcing programme ...,2919,1810


### Create new dataframe with sentence in each row 

In [45]:
df_sentence = pd.concat([pd.Series(row['brand_name'], row['message_text'].split('. '))
           for _, row in df.iterrows()]).reset_index()
df_sentence = df_sentence.rename(columns={'index': "message", 0 : 'brand'})

In [51]:
df_sentence = df_sentence[df_sentence['message'] != ' ']